In [49]:
import pandas as pd

In [50]:
df = pd.read_csv('../data/raw/movies_dataset_2000_2023.csv')

In [51]:
df.shape

(6095, 24)

In [52]:
df.columns

Index(['Unnamed: 0.1', 'Movie Title', 'Plot Summary', 'Cover Image', 'Year',
       'Source', 'Rated', 'omdb Year', 'omdb Title', 'Released', 'Runtime',
       'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country',
       'Awards', 'Poster', 'Ratings', 'imdbRating', 'BoxOffice', 'imdbID'],
      dtype='object')

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6095 entries, 0 to 6094
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  896 non-null    float64
 1   Movie Title   6095 non-null   object 
 2   Plot Summary  5556 non-null   object 
 3   Cover Image   5990 non-null   object 
 4   Year          6095 non-null   int64  
 5   Source        6095 non-null   object 
 6   Rated         841 non-null    object 
 7   omdb Year     871 non-null    object 
 8   omdb Title    871 non-null    object 
 9   Released      868 non-null    object 
 10  Runtime       861 non-null    object 
 11  Genre         870 non-null    object 
 12  Director      850 non-null    object 
 13  Writer        857 non-null    object 
 14  Actors        867 non-null    object 
 15  Plot          862 non-null    object 
 16  Language      866 non-null    object 
 17  Country       869 non-null    object 
 18  Awards        756 non-null  

In [54]:
!pip install nltk

In [55]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hemantrajsingh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hemantrajsingh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [56]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [57]:
def preprocess_text(text):
  # Remove HTML tags and special characters
    clean_text = re.sub('<.*?>', '', text)
    clean_text = re.sub(r'[^a-zA-Z\s]', '', clean_text)

    # Tokenization
    tokens = word_tokenize(clean_text)

    # Remove stop words and convert to lowercase
    stop_words = set(stopwords.words('english'))
    tokens = [token.lower() for token in tokens if token.lower() not in stop_words]

    return ' '.join(tokens)

In [64]:
df = df.dropna(subset=['Plot Summary'])

In [65]:
df['Plot Summary'].isna().sum()

0

In [66]:
df['ProcessedPlot'] = df['Plot Summary'].apply(preprocess_text)

In [67]:
# Save the preprocessed data
preprocessed_path = '../data/processed/preprocessed_dataset.csv'
df.to_csv(preprocessed_path, index=False)